In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
from nltk.stem import *
import re # regular expression

### 1.Tìm hiểu thuộc tính

In [2]:
data=pd.read_csv("data.csv")
data.head()

,name,ranked,score,popularity,favorite,published,genre,description,serialization,reading,completed,dropped,plan_to_read
0,Berserk,1,9.47,1,"122,974","Aug 25, 1989","Action, Adventure, Award Winning, Drama, Fanta...","Guts, a former mercenary now known as the ""Bla...",Young Animal,"364,280","87,307","10,263","154,118"
1,JoJo no Kimyou na Bouken Part 7: Steel Ball Run,2,9.30,26,"42,935","Jan 19, 2004","Action, Adventure, Mystery, Supernatural","In the American Old West, the world's greatest...",Ultra Jump,"34,922","162,358","2,141","50,652"
2,Vagabond,3,9.24,15,"40,221","Sep 3, 1998","Action, Adventure, Award Winning","In 16th-century Japan, Shinmen Takezou is a wi...",Morning,"108,564","81,383","5,054","145,679"
3,One Piece,4,9.22,3,"114,647","Jul 22, 1997","Action, Adventure, Fantasy","Gol D. Roger, a man referred to as the ""King o...",Shounen Jump (Weekly),"465,243","30,242","18,817","46,908"
4,Monster,5,9.15,29,"20,528","Dec 5, 1994","Award Winning, Drama, Mystery","Kenzou Tenma, a renowned Japanese neurosurgeon...",Big Comic Original,"33,413","94,357","2,754","97,297"


In [3]:
data.dtypes

name              object
ranked             int64
score            float64
popularity         int64
favorite          object
published         object
genre             object
description       object
serialization     object
reading           object
completed         object
dropped           object
plan_to_read      object
dtype: object

##### Ranked
Hạng của manga dựa trên công thức sau:

Weighted Score = (v / (v + m)) * S + (m / (v + m)) * C

- S: Average score for the anime/manga
- v: Number of users giving a score for the anime/manga
- m: Minimum number of scored users required to get a calculated score
- C: The mean score across the anime/manga database 

##### Score
Điểm số của manga được đánh giá trên thang 10

##### Popularity
Hạng độ phổ biến của manga dựa tổng người đọc, đang đọc, drop, hay dự định đọc.

##### Favorite
Số người đưa vào manga ưa thích

##### Published
Thời gian phát hành

##### Genre
Thể loại, có 18 thể loại gồm Action, Adventure, Avant Garde, Award Winning, Boys Love, Comedy, Drama, Fantasy, Girls Love, Gourmet, Horror, Mystery, Romance, Sci-Fi, Slice of Life, Sports, Supernatural, Suspense 

##### Description
Mô tả sơ lược về nội dung manga (có thể none)

##### Serialization
Được đăng dài kì ở tạp chí nào 

##### Reading
Số người đang đọc 

##### Completed
Số người đã đọc xong

##### Dropped
Số người quyết định bỏ ngang không đọc tiếp

##### Plan to Read
Số người dự định đọc

##### Plan to Read
Số người dự định đọc

### 2.Xử lí giá trị NaN

In [4]:
data.columns[data.isnull().any()].tolist() #xác định những cột có giá trị nan

['genre', 'description', 'serialization']

In [5]:
dump=data.copy()

In [6]:
dump[dump[['genre','description','serialization']].isnull().any(axis=1)] #finding nan in dataframe

,name,ranked,score,popularity,favorite,published,genre,description,serialization,reading,completed,dropped,plan_to_read
55,Veil,56,8.62,2730,363,"Nov 15, 2019","Drama, Romance, Slice of Life","Entirely in color, this manga is a representat...",NaN,"2,698",368,67,"3,771"
102,Chikan Otoko,103,8.49,339,"2,887",2004,"Comedy, Drama, Romance, Slice of Life","Molester Man is a perverted, socially inept ot...",NaN,"3,547","26,751","1,030","14,222"
126,Doraemon,127,8.44,1318,892,Dec 1969,"Adventure, Award Winning, Comedy, Sci-Fi, Slic...",Doraemon is cat shaped robot from the future t...,NaN,"2,167","6,501",916,"3,516"
141,Onanie Master Kurosawa,142,8.41,80,"10,236",Sep 2007,Drama,Fourteen-year-old Kakeru Kurosawa is an antiso...,NaN,"7,653","84,124","1,707","37,260"
192,Kininatteru Hito ga Otoko ja Nakatta,193,8.32,1678,499,"Apr 19, 2023",Girls Love,Kininatteru Hito ga Otoko ja Nakatta depicts t...,NaN,"7,563",162,141,"3,158"
229,Blue Giant Supreme,230,8.29,5829,26,"Sep 10, 2016",NaN,NaN,Big Comic,838,368,28,"1,866"
320,Getter Robo Saga,321,8.18,7884,180,Not available,Action,A compilation of the long-running Getter Robo ...,NaN,225,822,29,"1,112"
356,Doukyuusei: Gold Ring & Silver Ring,357,8.15,2529,52,"Nov 5, 2016",Boys Love,"Differences aside, Hikaru Kusakabe and Rihito ...",NaN,221,"5,359",45,"2,062"
360,Mahou Shoujo Madoka★Magica: The Different Story,361,8.15,1120,669,"Jun 12, 2012","Drama, Horror",Ninth-grader Mami Tomoe is not your average mi...,NaN,"1,357","9,450",189,"4,551"
390,Sabishisugite Lesbian Fuuzoku ni Ikimashita Re...,391,8.12,328,"2,619","Jun 17, 2016",Slice of Life,After her life takes a turn for worse shortly ...,NaN,"2,764","32,062",325,"12,599"


**Thường thì một tạp chí sẽ có những thể loại nhất định, liệu có thể dựa vào tạp chí để điền thể loại cho dữ liệu khuyết?**

**Giải pháp đề ra: Dùng thể loại xuất hiện nhiều nhất trong cùng một serialization để điền giá trị khuyết cho genre đối với serialization Anonymous thì ta thay thế bằng thể loại xuất hiện nhiều nhất trong data tức comedy**

**Còn về description thì ta sẽ dùng một câu thay thế là:No synopsis information has been added to this title.**

**Về serialization thì ta thay thế bằng Anonymous**

In [7]:
dump['description'].fillna("No synopsis information has been added to this title.",inplace=True)

In [8]:
dump['serialization'].fillna("Anonymous",inplace=True)

In [9]:
dump['genre']=dump.groupby('serialization')['genre'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else "Comedy"))

### 3.Chuyển đổi data type của column thành dữ liệu thích hợp

#### Xử lí dữ liệu continous

In [10]:
dump.dtypes

name              object
ranked             int64
score            float64
popularity         int64
favorite          object
published         object
genre             object
description       object
serialization     object
reading           object
completed         object
dropped           object
plan_to_read      object
dtype: object

In [11]:
dump[['favorite','reading','completed','dropped','plan_to_read']] = dump[['favorite','reading','completed','dropped','plan_to_read']].replace({',': ''}, regex=True)

In [12]:
dump.head()

,name,ranked,score,popularity,favorite,published,genre,description,serialization,reading,completed,dropped,plan_to_read
0,Berserk,1,9.47,1,122974,"Aug 25, 1989","Action, Adventure, Award Winning, Drama, Fanta...","Guts, a former mercenary now known as the ""Bla...",Young Animal,364280,87307,10263,154118
1,JoJo no Kimyou na Bouken Part 7: Steel Ball Run,2,9.30,26,42935,"Jan 19, 2004","Action, Adventure, Mystery, Supernatural","In the American Old West, the world's greatest...",Ultra Jump,34922,162358,2141,50652
2,Vagabond,3,9.24,15,40221,"Sep 3, 1998","Action, Adventure, Award Winning","In 16th-century Japan, Shinmen Takezou is a wi...",Morning,108564,81383,5054,145679
3,One Piece,4,9.22,3,114647,"Jul 22, 1997","Action, Adventure, Fantasy","Gol D. Roger, a man referred to as the ""King o...",Shounen Jump (Weekly),465243,30242,18817,46908
4,Monster,5,9.15,29,20528,"Dec 5, 1994","Award Winning, Drama, Mystery","Kenzou Tenma, a renowned Japanese neurosurgeon...",Big Comic Original,33413,94357,2754,97297


In [13]:
dump=dump.astype({"favorite":'int64', "reading":"int64", "completed":"int64", "dropped":"int64","plan_to_read":"int64"})
dump.dtypes

name              object
ranked             int64
score            float64
popularity         int64
favorite           int64
published         object
genre             object
description       object
serialization     object
reading            int64
completed          int64
dropped            int64
plan_to_read       int64
dtype: object

In [14]:
dump.loc[dump['published']=='Not available','published']='April 7, 1974' # chỉ có 1 dữ liệu thiếu nên ta tra google cho dữ liệu này

In [15]:
dump['published'] = pd.to_datetime(dump['published'], format='mixed')
dump.head()

,name,ranked,score,popularity,favorite,published,genre,description,serialization,reading,completed,dropped,plan_to_read
0,Berserk,1,9.47,1,122974,1989-08-25,"Action, Adventure, Award Winning, Drama, Fanta...","Guts, a former mercenary now known as the ""Bla...",Young Animal,364280,87307,10263,154118
1,JoJo no Kimyou na Bouken Part 7: Steel Ball Run,2,9.30,26,42935,2004-01-19,"Action, Adventure, Mystery, Supernatural","In the American Old West, the world's greatest...",Ultra Jump,34922,162358,2141,50652
2,Vagabond,3,9.24,15,40221,1998-09-03,"Action, Adventure, Award Winning","In 16th-century Japan, Shinmen Takezou is a wi...",Morning,108564,81383,5054,145679
3,One Piece,4,9.22,3,114647,1997-07-22,"Action, Adventure, Fantasy","Gol D. Roger, a man referred to as the ""King o...",Shounen Jump (Weekly),465243,30242,18817,46908
4,Monster,5,9.15,29,20528,1994-12-05,"Award Winning, Drama, Mystery","Kenzou Tenma, a renowned Japanese neurosurgeon...",Big Comic Original,33413,94357,2754,97297


In [16]:
dump['published']=dump['published'].dt.year

In [17]:
dump.head()

,name,ranked,score,popularity,favorite,published,genre,description,serialization,reading,completed,dropped,plan_to_read
0,Berserk,1,9.47,1,122974,1989,"Action, Adventure, Award Winning, Drama, Fanta...","Guts, a former mercenary now known as the ""Bla...",Young Animal,364280,87307,10263,154118
1,JoJo no Kimyou na Bouken Part 7: Steel Ball Run,2,9.30,26,42935,2004,"Action, Adventure, Mystery, Supernatural","In the American Old West, the world's greatest...",Ultra Jump,34922,162358,2141,50652
2,Vagabond,3,9.24,15,40221,1998,"Action, Adventure, Award Winning","In 16th-century Japan, Shinmen Takezou is a wi...",Morning,108564,81383,5054,145679
3,One Piece,4,9.22,3,114647,1997,"Action, Adventure, Fantasy","Gol D. Roger, a man referred to as the ""King o...",Shounen Jump (Weekly),465243,30242,18817,46908
4,Monster,5,9.15,29,20528,1994,"Award Winning, Drama, Mystery","Kenzou Tenma, a renowned Japanese neurosurgeon...",Big Comic Original,33413,94357,2754,97297


#### Xử lý dữ liệu categorical

##### Xử lý genre

In [18]:
# Assuming 'df' is your DataFrame and 'column_name' is the column with comma-separated classes
split_df = dump['genre'].str.split(', ', expand=True).stack()
split_df = split_df.reset_index(level=1, drop=True).rename('genre')
df = dump.drop('genre', axis=1).join(split_df)
df.head()


,name,ranked,score,popularity,favorite,published,description,serialization,reading,completed,dropped,plan_to_read,genre
0,Berserk,1,9.47,1,122974,1989,"Guts, a former mercenary now known as the ""Bla...",Young Animal,364280,87307,10263,154118,Action
0,Berserk,1,9.47,1,122974,1989,"Guts, a former mercenary now known as the ""Bla...",Young Animal,364280,87307,10263,154118,Adventure
0,Berserk,1,9.47,1,122974,1989,"Guts, a former mercenary now known as the ""Bla...",Young Animal,364280,87307,10263,154118,Award Winning
0,Berserk,1,9.47,1,122974,1989,"Guts, a former mercenary now known as the ""Bla...",Young Animal,364280,87307,10263,154118,Drama
0,Berserk,1,9.47,1,122974,1989,"Guts, a former mercenary now known as the ""Bla...",Young Animal,364280,87307,10263,154118,Fantasy


**Base on number manga of genre i can using below mapping: "Action (9,188)
Adventure (4,019)
Avant Garde (73)
Award Winning (439)
Boys Love (9,905)
Comedy (14,408)
Drama (9,580)
Fantasy (11,383)
Girls Love (1,652)
Gourmet (441)
Horror (1,726)
Mystery (2,492)
Romance (17,244)
Sci-Fi (3,291)
Slice of Life (5,216)
Sports (1,153)
Supernatural (7,652)
Suspense (560)"**

In [19]:
mapping1 = { 'Romance': 18, 
    'Comedy': 17, 
    'Action': 16, 
    'Boys Love': 15, 
    'Drama': 14, 
    'Fantasy': 13, 
    'Supernatural': 12, 
    'Slice of Life': 11, 
    'Adventure': 10, 
    'Sci-Fi': 9, 
    'Mystery': 8, 
    'Horror': 7, 
    'Girls Love': 6, 
    'Sports': 5, 
    'Suspense': 4, 
    'Award Winning': 3, 
    'Gourmet': 2, 
    'Avant Garde': 1}
df['genre']=df['genre'].map(mapping1)


In [20]:
df=df.groupby(level=0).min()
df.head()

,name,ranked,score,popularity,favorite,published,description,serialization,reading,completed,dropped,plan_to_read,genre
0,Berserk,1,9.47,1,122974,1989,"Guts, a former mercenary now known as the ""Bla...",Young Animal,364280,87307,10263,154118,3.0
1,JoJo no Kimyou na Bouken Part 7: Steel Ball Run,2,9.30,26,42935,2004,"In the American Old West, the world's greatest...",Ultra Jump,34922,162358,2141,50652,8.0
2,Vagabond,3,9.24,15,40221,1998,"In 16th-century Japan, Shinmen Takezou is a wi...",Morning,108564,81383,5054,145679,3.0
3,One Piece,4,9.22,3,114647,1997,"Gol D. Roger, a man referred to as the ""King o...",Shounen Jump (Weekly),465243,30242,18817,46908,10.0
4,Monster,5,9.15,29,20528,1994,"Kenzou Tenma, a renowned Japanese neurosurgeon...",Big Comic Original,33413,94357,2754,97297,3.0


##### Xử lý serialization

In [21]:
label_encoder = LabelEncoder()
df['serialization'] = label_encoder.fit_transform(df['serialization']) #range 0->n_class-1


In [22]:
df.head()

,name,ranked,score,popularity,favorite,published,description,serialization,reading,completed,dropped,plan_to_read,genre
0,Berserk,1,9.47,1,122974,1989,"Guts, a former mercenary now known as the ""Bla...",221,364280,87307,10263,154118,3.0
1,JoJo no Kimyou na Bouken Part 7: Steel Ball Run,2,9.30,26,42935,2004,"In the American Old West, the world's greatest...",210,34922,162358,2141,50652,8.0
2,Vagabond,3,9.24,15,40221,1998,"In 16th-century Japan, Shinmen Takezou is a wi...",161,108564,81383,5054,145679,3.0
3,One Piece,4,9.22,3,114647,1997,"Gol D. Roger, a man referred to as the ""King o...",189,465243,30242,18817,46908,10.0
4,Monster,5,9.15,29,20528,1994,"Kenzou Tenma, a renowned Japanese neurosurgeon...",19,33413,94357,2754,97297,3.0


##### Xử lí description

In [23]:
df['description']

0      Guts, a former mercenary now known as the "Bla...
1      In the American Old West, the world's greatest...
2      In 16th-century Japan, Shinmen Takezou is a wi...
3      Gol D. Roger, a man referred to as the "King o...
4      Kenzou Tenma, a renowned Japanese neurosurgeon...
                             ...                        
995    A series of one-shots that ran in Monthly Melo...
996    Energetic and strong-willed, Beni doesn't care...
997    Frau and Kio are angels, and it's their duty t...
998    Shanaou Yoshitsune is an alternative story to ...
999    Touma Kamijou just can't catch a break. He mig...
Name: description, Length: 1000, dtype: object

In [24]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))
porter_stemmer = PorterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()


def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()

    # Replace weird characters with spaces
    text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)

    # Tokenization
    words = text.split()

    # Stemming and Lemmatization, and remove stopwords
    words = [porter_stemmer.stem(word) for word in words if word not in stop_words]
    words = [wordnet_lemmatizer.lemmatize(word) for word in words]

    # Join the processed words back into a sentence
    text = ' '.join(words)

    return text

df['description'] = df['description'].apply(preprocess_text)


#### Tìm hiểu phân bố giá trị của dữ liệu

In [25]:
def missing_ratio(x):
    return round((x.isnull().sum() / len(x)) * 100, 1)

def lower_quartile(x):
    return round(x.quantile(0.25), 1)

def median(x):
    return round(x.median(), 1)

def upper_quartile(x):
    return round(x.quantile(0.75), 1)
num_col_info_df=df.select_dtypes(include=np.number)
num_col_info_df = num_col_info_df.agg([missing_ratio, "min", lower_quartile, median, upper_quartile, "max"]).round(1)
num_col_info_df

,ranked,score,popularity,favorite,published,serialization,reading,completed,dropped,plan_to_read,genre
missing_ratio,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,1.0,7.8,1.0,1.0,1966.0,0.0,18.0,24.0,5.0,126.0,2.0
lower_quartile,250.8,7.9,373.8,150.8,2003.0,61.8,1170.5,1028.0,143.8,3015.2,7.0
median,500.5,8.0,1040.5,439.0,2009.0,130.5,3266.0,3805.0,434.5,6738.0,11.0
upper_quartile,750.2,8.3,2610.8,1697.0,2015.0,189.0,9729.8,11020.8,1237.8,15015.5,14.0
max,1000.0,9.5,26745.0,122974.0,2023.0,243.0,465243.0,302871.0,26342.0,154118.0,18.0


### 4. Lưu vào file csv

In [26]:
df.to_csv('preprocessed_data.csv',index=False)